# 날씨 데이터 스크래핑

In [5]:
import requests

from  bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Mostly clear, with a low around 50. North wind around 6 mph becoming calm. " class="forecast-icon" src="newimages/medium/nfew.png" title="Overnight: Mostly clear, with a low around 50. North wind around 6 mph becoming calm. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 50 °F
 </p>
</div>


In [7]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Overnight
Mostly Clear
Low: 50 °F


In [10]:
img = tonight.find("img")
desc = img['title']
print(desc)

Overnight: Mostly clear, with a low around 50. North wind around 6 mph becoming calm. 


모든 정보 추출하기

In [12]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight']

In [15]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Sunny', 'Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Mostly Clear', 'Sunny', 'Clear']
['Low: 50 °F', 'High: 78 °F', 'Low: 54 °F', 'High: 84 °F', 'Low: 54 °F', 'High: 75 °F', 'Low: 51 °F', 'High: 69 °F', 'Low: 49 °F']
['Overnight: Mostly clear, with a low around 50. North wind around 6 mph becoming calm. ', 'Wednesday: Sunny, with a high near 78. Light and variable wind becoming west 10 to 15 mph in the afternoon. Winds could gust as high as 20 mph. ', 'Wednesday Night: Clear, with a low around 54. West wind 11 to 16 mph becoming light and variable  in the evening. Winds could gust as high as 21 mph. ', 'Thursday: Sunny, with a high near 84. North wind 3 to 6 mph. ', 'Thursday Night: Mostly clear, with a low around 54. West southwest wind 5 to 14 mph, with gusts as high as 18 mph. ', 'Friday: Mostly sunny, with a high near 75.', 'Friday Night: Mostly clear, with a low around 51.', 'Saturday: Sunny, with a high near 69.', 'Saturday Night: Clear, with a low around 

모든 데이터를 Pandas Dataframe으로 합치기

In [16]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Overnight,Mostly Clear,Low: 50 °F,"Overnight: Mostly clear, with a low around 50...."
1,Wednesday,Sunny,High: 78 °F,"Wednesday: Sunny, with a high near 78. Light a..."
2,WednesdayNight,Clear,Low: 54 °F,"Wednesday Night: Clear, with a low around 54. ..."
3,Thursday,Sunny,High: 84 °F,"Thursday: Sunny, with a high near 84. North wi..."
4,ThursdayNight,Mostly Clear,Low: 54 °F,"Thursday Night: Mostly clear, with a low aroun..."
5,Friday,Mostly Sunny,High: 75 °F,"Friday: Mostly sunny, with a high near 75."
6,FridayNight,Mostly Clear,Low: 51 °F,"Friday Night: Mostly clear, with a low around 51."
7,Saturday,Sunny,High: 69 °F,"Saturday: Sunny, with a high near 69."
8,SaturdayNight,Clear,Low: 49 °F,"Saturday Night: Clear, with a low around 49."


In [19]:
temp_nums = weather["temp"].str.extract(r'(?P<digit>\d+)', expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    50
1    78
2    54
3    84
4    54
5    75
6    51
7    69
8    49
Name: digit, dtype: object

In [21]:
weather["temp_num"].mean()

62.666666666666664

In [23]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [24]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
0,Overnight,Mostly Clear,Low: 50 °F,"Overnight: Mostly clear, with a low around 50....",50,True
2,WednesdayNight,Clear,Low: 54 °F,"Wednesday Night: Clear, with a low around 54. ...",54,True
4,ThursdayNight,Mostly Clear,Low: 54 °F,"Thursday Night: Mostly clear, with a low aroun...",54,True
6,FridayNight,Mostly Clear,Low: 51 °F,"Friday Night: Mostly clear, with a low around 51.",51,True
8,SaturdayNight,Clear,Low: 49 °F,"Saturday Night: Clear, with a low around 49.",49,True
